In [760]:
%run ../config/config.py
#from config.config import *
import pandas as pd
import numpy as np
import debug
import cv2

In [761]:
image = cv2.imread(BLACK_YELLOW)
image_size = (SIZE,SIZE)
image = cv2.resize(image,image_size)

In [762]:
def _detect_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, THRESHOLD1, THRESHOLD2, EDGES)
    return edges

edges_image = _detect_edges(image)

In [763]:
def _generate_edges_lines(canny):
    
    lines = cv2.HoughLines(canny, 
                            rho = RHO_RESOLUTION, 
                            theta =  THETA_RESOLUTION, 
                            threshold = HOUGH_THRESHOLD)
    
    #Format dataset
    lines = lines.reshape(len(lines),2)
    
    coor_lines = []
    for line in lines:
        rho,theta = line
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 +DISTANCE*(-b))
        y1 = int(y0 +DISTANCE*(a))
        x2 = int(x0 -DISTANCE*(-b))
        y2 = int(y0 -DISTANCE*(a))   
        coor_lines.append(((x1,y1),(x2,y2)))
    
    return np.array(coor_lines)
    
lines = _generate_edges_lines(edges_image)

In [764]:
from sklearn.cluster import KMeans

def _get_lines_means(points, direction, n_clusters = 9):
    
    means = []

    for lines in points:
    
        kmeans = KMeans(n_clusters = n_clusters, n_init=10)
        kmeans.fit(lines)
        labels = kmeans.labels_
    
        unique_labels = list(range(n_clusters))
    
        cluster_means = []
    
        for label in unique_labels: 
            cluster_data = lines[labels == label] 
            cluster_mean = np.round(sum(cluster_data)/len(cluster_data))  
            cluster_means.append(cluster_mean)
        
        cluster_means = np.array(cluster_means)
        sorted_index = np.argsort(cluster_means[:, direction])
        
        means.append(cluster_means[sorted_index])
 
    return means


In [765]:


def _get_vertical_averages(lines):
    
    vertical_lines = lines[lines[:,0,1] > SIZE]
    
    initial_points = vertical_lines[:,1][vertical_lines[:,1,1] < -SIZE]
    final_points = vertical_lines[:,0][vertical_lines[:,0,1] > SIZE] 
    
    averages = _get_lines_means((initial_points, final_points), VERTICAL)
    
    return np.stack(averages, axis=1)

def _get_horizontal_averages(lines):

    horizontal_lines = lines[lines[:,1,0] > SIZE]
    
    initial_points = horizontal_lines[:,0][horizontal_lines[:,0,0] < -SIZE]
    final_points = horizontal_lines[:,1][horizontal_lines[:,1,0] > SIZE]
    
    averages = _get_lines_means((initial_points, final_points), HORIZONTAL)
    
    return np.stack(averages, axis=1)
    
vertical_lines = _get_vertical_averages(lines).astype(int)
horizontal_lines = _get_horizontal_averages(lines).astype(int)

lines = np.concatenate((vertical_lines,horizontal_lines), axis=0)

debug.lines(image,lines, size = 3, color=GREEN);


In [766]:
def _detect_inner_corners(lines, corners = MAX_CORNERS):
    
    black_image = np.zeros((SIZE, SIZE, EDGES), dtype=np.uint8)
    debug.lines(black_image,lines, size = 1);
    black_image = _detect_edges(black_image)
    points = cv2.goodFeaturesToTrack(black_image, corners, QUALITY, MIN_DISTANCE)
    return points.reshape(len(points),2)
#debug.lines(image,lines, size = 1);
points =  _detect_inner_corners(lines)

points;


In [767]:
def _get_middle_squares(lines):

    middle_lines = []

    for i in range(17):
        
        if i == 8: continue
        
        mean = (lines[i] +lines[i+1])/2
        
        middle_line = mean.astype(int).tolist()
        middle_lines.append(middle_line)
        
    corners = _detect_inner_corners(middle_lines, 64)
    return corners

middle_points = _get_middle_squares(lines)


In [768]:
def _distance(a, b):
    
    x_diff = abs(a[0] - b[0])
    y_diff = abs(a[1] - b[1])
    
    distance = np.sqrt(x_diff**2 + y_diff**2)
    
    return distance

In [769]:
def _get_grid(middle_points, lines):
    grid_points = _detect_inner_corners(lines)
    
    grid = []
    
    for middle_point in middle_points:
        
        distances = []
        
        for point in grid_points:
            
            distances.append(_distance(middle_point, point))
            
        index = np.argsort(distances)
        
        order = grid_points[index]
        
        final = order[:4].tolist()
        
        grid.append(final)
        
    #grid_index = np.argsort(middle_point)
    return grid
    #print(_distance(points, grid_points))

def ordenar_puntos(punto):
    x, y = punto
    return (y, x)

# Utiliza la función sorted para ordenar los puntos
middle_points = sorted(middle_points, key = ordenar_puntos)

grid = _get_grid(middle_points, lines)

#debug.points(image, middle_points, color = RED, size = 8)


In [770]:

def line_intersection(line1, line2):
    line1 = line1.tolist()
    line2 = line2.tolist()
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return (x, y)

    # Muestra la imagen con las líneas y la intersección


for i in range(9):
    for j in range(9,18):
        res = line_intersection(lines[i], lines[j])
        cv2.circle(image,center=(int(res[0]),int(res[1])),radius=10,color=RED, thickness= -1)  


print(res)

(656.5534079951224, 639.4706028955877)


In [771]:

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()